In [1]:
%cd ../

/home/dungmaster/MLProjects/face_recognition/AI-Checkin


In [23]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import os
import torch
import glob
import os.path as osp
from sklearn.metrics.pairwise import cosine_similarity
from torchvision import datasets, transforms
from PIL import Image
from tqdm import tqdm
import numpy as np
import pickle
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
nc = glob.glob('/home/dungmaster/MLProjects/face_recognition/face_aic/*/*')
nc = [osp.basename(n) for n in nc]
nc = set(nc)
len(nc)


126

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [5]:
val_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

# Pretrained Model

In [6]:
resnet = InceptionResnetV1(
    classify=False,
    pretrained='vggface2',
    num_classes=nc
).to(device)
resnet.eval()

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [7]:
def _save_pickle(obj, file_path):
  with open(file_path, 'wb') as f:
    pickle.dump(obj, f)

def _load_pickle(file_path):
  with open(file_path, 'rb') as f:
    obj = pickle.load(f)
  return obj

In [8]:
def _embed_image(path, model):
    image = Image.open(path)
    image = val_transform(image).unsqueeze(0).to(device)
    emb = model(image).detach().cpu().numpy()
    return emb

In [9]:
# emb_vecs = []
# labels = []

# for image_path in tqdm(glob.glob('/home/dungmaster/MLProjects/face_recognition/face_aic/train/*/*.jpg')):
#     label = image_path.split('/')[-2]
#     emb = _embed_image(image_path, resnet)
#     emb_vecs.append(emb)
#     labels.append(label)

100%|██████████| 509/509 [00:21<00:00, 23.99it/s]


In [11]:
# _save_pickle(emb_vecs, '../data/embed_faces.pkl')
# _save_pickle(labels, "../data/labels.pkl")

In [12]:
embed_faces = np.stack(_load_pickle("../data/embed_faces.pkl"))
embed_faces = np.squeeze(embed_faces, axis=1)
y_labels = _load_pickle("../data/labels.pkl")

In [13]:
def _most_similarity(embed_vecs, vec, labels):
  sim = cosine_similarity(embed_vecs, vec)
  sim = np.squeeze(sim, axis = 1)
  argmax = np.argsort(sim)[::-1][:1]
  label = [labels[idx] for idx in argmax][0]
  return label

vec = _embed_image('/home/dungmaster/MLProjects/face_recognition/face_aic/val/Đặng Viết Tùng/5.jpg', resnet)
_most_similarity(embed_faces, vec, y_labels)

'Đặng Viết Tùng'

In [14]:
y_preds = []
y_gt = []

for image_path in tqdm(glob.glob('/home/dungmaster/MLProjects/face_recognition/face_aic/val/*/*.jpg')):
  label = image_path.split('/')[-2]
  vec = _embed_image(image_path, resnet)
  y_pred = _most_similarity(embed_faces, vec, y_labels)
  y_preds.append(y_pred)
  y_gt.append(label)

print(accuracy_score(y_preds, y_gt))

100%|██████████| 20/20 [00:02<00:00,  7.92it/s]

0.55


In [15]:
y_preds

['Nguyễn Đức Huy',
 'Nguyễn Ngọc Phương Linh',
 'Dương Thị Thùy Linh',
 'Nguyễn Trung Thành',
 'Nguyễn Phương Nguyên',
 'Trần Duy Quốc Huy',
 'Dương Quốc Khánh',
 'Trịnh Đức Linh',
 'Ngô Anh Kiệt',
 'Nguyễn Long Dương',
 'Bùi Minh Trang',
 'Nguyễn Duy Minh',
 'Vũ Hoàng Tài Toàn',
 'Đặng Viết Tùng',
 'Phạm Ánh Nguyệt',
 'Trần Đức Lâm',
 'Lý Đức Hùng',
 'Nguyễn Đức Hoàng Lâm',
 'Trần Mạnh Tiến',
 'Hoàng Anh Đức']

# ArcFace Model

In [16]:
resnet.load_state_dict(torch.load('/home/dungmaster/MLProjects/face_recognition/arcface.pth'))

<All keys matched successfully>

In [17]:
emb_vecs = []
labels = []

for image_path in tqdm(glob.glob('/home/dungmaster/MLProjects/face_recognition/face_aic/train/*/*.jpg')):
    label = image_path.split('/')[-2]
    emb = _embed_image(image_path, resnet)
    emb_vecs.append(emb)
    labels.append(label)

100%|██████████| 509/509 [00:19<00:00, 26.46it/s]


In [18]:
_save_pickle(emb_vecs, '../data/arc_embed_faces.pkl')
_save_pickle(labels, "../data/arc_labels.pkl")

In [19]:
embed_faces = np.stack(_load_pickle("../data/arc_embed_faces.pkl"))
embed_faces = np.squeeze(embed_faces, axis=1)
y_labels = _load_pickle("../data/arc_labels.pkl")

In [20]:
y_preds_1 = []
y_gt_1 = []

for image_path in tqdm(glob.glob('/home/dungmaster/MLProjects/face_recognition/face_aic/val/*/*.jpg')):
  label = image_path.split('/')[-2]
  vec = _embed_image(image_path, resnet)
  y_pred = _most_similarity(embed_faces, vec, y_labels)
  y_preds_1.append(y_pred)
  y_gt_1.append(label)

print(accuracy_score(y_preds_1, y_gt_1))

100%|██████████| 20/20 [00:02<00:00,  8.60it/s]

0.55


In [21]:
y_preds_1 

['Nguyễn Đức Huy',
 'Nguyễn Ngọc Phương Linh',
 'Nguyễn Phương Nguyên',
 'Ngô Anh Kiệt',
 'Nguyễn Phương Nguyên',
 'Đào Xuân Bắc',
 'Bùi Mộc Quế Anh',
 'Trịnh Đức Linh',
 'Trương Thanh Bình',
 'Nguyễn Long Dương',
 'Hoàng Thị Lan Nhi',
 'Nguyễn Duy Minh',
 'Trần Mạnh Tiến',
 'Đặng Viết Tùng',
 'Phạm Ánh Nguyệt',
 'Nguyễn Tiến Quang',
 'Lý Đức Hùng',
 'Nguyễn Đức Hoàng Lâm',
 'Trần Mạnh Tiến',
 'Nguyễn Trung Thành']

In [22]:
y_preds

['Nguyễn Đức Huy',
 'Nguyễn Ngọc Phương Linh',
 'Dương Thị Thùy Linh',
 'Nguyễn Trung Thành',
 'Nguyễn Phương Nguyên',
 'Trần Duy Quốc Huy',
 'Dương Quốc Khánh',
 'Trịnh Đức Linh',
 'Ngô Anh Kiệt',
 'Nguyễn Long Dương',
 'Bùi Minh Trang',
 'Nguyễn Duy Minh',
 'Vũ Hoàng Tài Toàn',
 'Đặng Viết Tùng',
 'Phạm Ánh Nguyệt',
 'Trần Đức Lâm',
 'Lý Đức Hùng',
 'Nguyễn Đức Hoàng Lâm',
 'Trần Mạnh Tiến',
 'Hoàng Anh Đức']

In [ ]:
idx_diff = np.flatnonzero(np.array(y_preds) != np.array(y_test))

fg, ax = plt.subplots(2, 5, figsize=(15, 8))
fg.suptitle('Wrong predict images')

for i in np.arange(2):
  for j in np.arange(5):
    ax[i, j].imshow(X_test[idx_diff[i + j + j*i]])
    ax[i, j].set_xlabel('Transform '+str(i+j+j*i))
    ax[i, j].axis('off')